In [1]:
import sys
sys.path.append('/home/aarushg/KAN-FPGA/KAN_Impl')
from KANLinear import KAN, Quantizer
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


device = "cpu"

In [2]:
grid_range=[-4, 4]

input_dim = 1
output_dim = 2

In [3]:
# Load from files
X_train = np.load('data/X_train_val.npy')
y_train = np.load('data/y_train_val.npy')
X_test = np.load('data/X_test.npy')
y_test = np.load('data/y_test.npy')

model = KAN([1,2], grid_size=30, spline_order=10, grid_eps=0.05, base_activation=nn.GELU, grid_range=grid_range, quantize=True, tp=7, fp=3, lut_res=128).to(device)

# model = KAN([16,4,5], grid_size=30, spline_order=3, grid_eps=0.05, base_activation=nn.GELU, grid_range=grid_range).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
# Define learning rate scheduler
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
trainloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

training_loss = []
testing_loss = []

# Define loss
criterion = nn.CrossEntropyLoss()
for epoch in range(10):
    # Train
    model.train()
    epoch_train_loss = 0  # Initialize loss for the epoch
    total_batches = 0
    with tqdm(trainloader) as pbar:
        for i, (inputs, labels) in enumerate(pbar):
            inputs = inputs.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels.to(device))
            loss.backward()

            optimizer.step()

            epoch_train_loss += loss.item()
            total_batches += 1

            accuracy = (output.argmax(dim=1) == labels.to(device)).float().mean()

            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])
    
    average_train_loss = epoch_train_loss / total_batches
    training_loss.append(average_train_loss)  # Record the average training loss

TP:  7 FP:  3
LUT TP:  7 LUT FP:  3


 26%|██▌       | 26/100 [00:00<00:00, 258.22it/s, accuracy=0.5, loss=0.758, lr=0.0001]

Before quantizing:  tensor([[2.0641]])
After quantizing:  tensor([[2.]])
Before quantizing:  tensor([[-1.0023,  1.2978]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-1.1250,  1.2500]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.2866]])
After quantizing:  tensor([[0.2500]])
Before quantizing:  tensor([[-0.0767,  0.0994]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.1250,  0.0000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.1146]])
After quantizing:  tensor([[0.]])
Before quantizing:  tensor([[-1.8385e-05,  3.2574e-06]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.1250,  0.0000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[1.3495]])
After quantizing:  tensor([[1.2500]])
Before quantizing:  tensor([[-0.5730,  0.7421]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.6250,  0.6250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.5847]])
After quantizing:  tensor([[-0.6250]])
Before quantizing

100%|██████████| 100/100 [00:00<00:00, 263.16it/s, accuracy=0.5, loss=0.633, lr=0.0001]


Before quantizing:  tensor([[-0.9580]])
After quantizing:  tensor([[-1.]])
Before quantizing:  tensor([[ 0.0809, -0.1051]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.1158]])
After quantizing:  tensor([[0.]])
Before quantizing:  tensor([[ 8.4860e-05, -1.4924e-05]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.3992]])
After quantizing:  tensor([[0.3750]])
Before quantizing:  tensor([[-0.1237,  0.1605]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.1250,  0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.6215]])
After quantizing:  tensor([[0.5000]])
Before quantizing:  tensor([[-0.1766,  0.2290]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.2500,  0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.5091]])
After quantizing:  tensor([[0.5000]])
Before quantizing

...

After quantizing:  tensor([[-0.5000,  0.5000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.2866]])
After quantizing:  tensor([[0.2500]])
Before quantizing:  tensor([[-0.0721,  0.0953]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.1250,  0.0000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.7966]])
After quantizing:  tensor([[0.7500]])
Before quantizing:  tensor([[-0.2805,  0.3697]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.3750,  0.2500]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.8726]])
After quantizing:  tensor([[-0.8750]])
Before quantizing:  tensor([[ 0.0738, -0.1038]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.3504]])
After quantizing:  tensor([[-0.3750]])
Before quantizing:  tensor([[ 0.0606, -0.0833]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizin

 78%|███████▊  | 78/100 [00:00<00:00, 256.29it/s, accuracy=0.5, loss=0.633, lr=0.0001]

Before quantizing:  tensor([[1.0781]])
After quantizing:  tensor([[1.]])
Before quantizing:  tensor([[-0.4061,  0.5359]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.5000,  0.5000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.8126]])
After quantizing:  tensor([[-0.8750]])
Before quantizing:  tensor([[ 0.0738, -0.1038]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[1.4306]])
After quantizing:  tensor([[1.3750]])
Before quantizing:  tensor([[-0.6077,  0.8017]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.6250,  0.7500]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.1118]])
After quantizing:  tensor([[0.]])
Before quantizing:  tensor([[-2.6396e-04,  9.9600e-05]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.1250,  0.0000]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-1.5229]])
After quantizing:  tensor([[-1.6250]])
Before quantizi

100%|██████████| 100/100 [00:00<00:00, 256.98it/s, accuracy=0.5, loss=0.18, lr=0.0001]

After quantizing:  tensor([[-0.3750,  0.3750]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.9267]])
After quantizing:  tensor([[-1.]])
Before quantizing:  tensor([[ 0.0697, -0.0985]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-1.9476]])
After quantizing:  tensor([[-2.]])
Before quantizing:  tensor([[ 0.0238, -0.0296]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[0.5377]])
After quantizing:  tensor([[0.5000]])
Before quantizing:  tensor([[-0.1671,  0.2202]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.2500,  0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  tensor([[-0.2137]])
After quantizing:  tensor([[-0.2500]])
Before quantizing:  tensor([[ 0.0462, -0.0630]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
Before quantizing:  te

In [4]:
# Temporarily store quantizers
stored_quantizers = []
for layer in model.layers:
    layer_quantizers = {}
    if hasattr(layer, 'lut_inp_quantizer'):
        layer_quantizers['inp'] = layer.lut_inp_quantizer
        delattr(layer, 'lut_inp_quantizer')
    if hasattr(layer, 'lut_out_quantizer'):
        layer_quantizers['out'] = layer.lut_out_quantizer
        delattr(layer, 'lut_out_quantizer')
    stored_quantizers.append(layer_quantizers)

# Save model without quantizers
# torch.save(model, 'model.pth')

In [5]:
model = torch.load('model.pth')

# Restore quantizers
for layer, quantizers in zip(model.layers, stored_quantizers):
    if 'inp' in quantizers:
        layer.lut_inp_quantizer = quantizers['inp']
    if 'out' in quantizers:
        layer.lut_out_quantizer = quantizers['out']


/tmp/ipykernel_974561/1461645246.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model.pth')


In [12]:
file_contents = ""

for inp in torch.Tensor(X_test):
    res = model(inp).tolist()
    print(inp, "->", res)
    file_contents += f"{0 if res[0] == 0.0 else res[0]} {0 if res[1] == 0.0 else res[1]} \n"

with open("test_results.txt", "w") as f:
    f.write(file_contents)

Before quantizing:  tensor([[-1.5239]])
After quantizing:  tensor([[-1.6250]])
Before quantizing:  tensor([[ 0.0813, -0.0696]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[ 0.0000, -0.1250]], grad_fn=<QuantizeBackward>)
tensor([-1.5239]) -> [0.0, -0.125]
Before quantizing:  tensor([[0.7651]])
After quantizing:  tensor([[0.7500]])
Before quantizing:  tensor([[-0.4929,  0.4087]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.5000,  0.3750]], grad_fn=<QuantizeBackward>)
tensor([0.7651]) -> [-0.5, 0.375]
Before quantizing:  tensor([[1.1581]])
After quantizing:  tensor([[1.1250]])
Before quantizing:  tensor([[-0.8371,  0.6954]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-0.8750,  0.6250]], grad_fn=<QuantizeBackward>)
tensor([1.1581]) -> [-0.875, 0.625]
Before quantizing:  tensor([[2.6012]])
After quantizing:  tensor([[2.5000]])
Before quantizing:  tensor([[-2.1613,  1.8036]], grad_fn=<AddBackward0>)
After quantizing:  tensor([[-2.2500,  1.7500]], grad_fn=<Quantize